# Calibration

In [ ]:
import glob
import cv2
import numpy as np

# --- 3.1 Specify checkerboard parameters ---
CHECKERBOARD = (9, 6)        # inner corners per row, per column
SQUARE_SIZE  = 0.025         # square size in meters (e.g. 25 mm → 0.025 m)

# prepare object points: (0,0,0), (s,0,0), (2s,0,0), …, ( (m-1)s, (n-1)s, 0 )
objp = np.zeros((CHECKERBOARD[1]*CHECKERBOARD[0], 3), np.float32)
objp[:, :2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)
objp *= SQUARE_SIZE

# storage for 3D points (world) and 2D points (image)
objpoints = []  # 3D points in world
imgpoints = []  # 2D points in image

# --- 3.2 Detect corners in all images ---
images = glob.glob('calib_*.jpg')
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # find chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD,
                                             cv2.CALIB_CB_ADAPTIVE_THRESH
                                           | cv2.CALIB_CB_NORMALIZE_IMAGE)
    if ret:
        # refine corner locations
        corners2 = cv2.cornerSubPix(gray, corners, (11,11), (-1,-1),
                                    criteria=(cv2.TERM_CRITERIA_EPS 
                                              | cv2.TERM_CRITERIA_MAX_ITER,
                                              30, 0.001))
        objpoints.append(objp)
        imgpoints.append(corners2)
        # (optional) draw and display:
        cv2.drawChessboardCorners(img, CHECKERBOARD, corners2, ret)
        cv2.imshow('Corners', img); cv2.waitKey(100)
cv2.destroyAllWindows()

# --- 3.3 Calibrate camera ---
ret, K, dist, rvecs, tvecs = cv2.calibrateCamera(
    objpoints, imgpoints, gray.shape[::-1], None, None)

print("Re-projection error:", ret)
print("Camera matrix K:\n", K)
print("Distortion coeffs:\n", dist.ravel())

# --- 3.4 Example: compute extrinsics for image #0 ---
# rotation vector → rotation matrix
R0, _ = cv2.Rodrigues(rvecs[0])
t0 = tvecs[0]   # shape (3,1)
print("R0:\n", R0)
print("t0:\n", t0)


# Pose Estimation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 1) Define a small 3D grid
Nx, Ny, Nz = 20, 20, 20
x = np.linspace(-1, 1, Nx)
y = np.linspace(-1, 1, Ny)
z = np.linspace(0, 1, Nz)

# 2) Create two synthetic "silhouettes" (masks) for each camera:
#    - left camera sees projection along X → silhouette in Y-Z plane
#    - right camera sees projection along Y → silhouette in X-Z plane
radius = 0.5
left_silhouette  = np.abs(y)[:, None] <= radius    # shape (Ny,1)
right_silhouette = np.abs(x)[None, :] <= radius    # shape (1,Nx)

# 3) Voxel carving: keep (x,y,z) if both silhouettes agree
points = []
for i, xi in enumerate(x):
    for j, yj in enumerate(y):
        if right_silhouette[0, i] and left_silhouette[j, 0]:
            for zk in z:
                points.append([xi, yj, zk])
points = np.array(points)

# 4) Compute a simple centerline: for each z-slice, average x & y
centerline = []
for zk in z:
    slice_pts = points[np.isclose(points[:, 2], zk)]
    if len(slice_pts) > 0:
        cx, cy = slice_pts[:,0].mean(), slice_pts[:,1].mean()
        centerline.append([cx, cy, zk])
centerline = np.array(centerline)

# 5) Estimate tangent at the tip (last two points)
v = centerline[-1] - centerline[-2]
theta = np.arccos(v[2] / np.linalg.norm(v))  # bending magnitude
phi   = np.arctan2(v[1], v[0])              # bending plane

print("Estimated bending angle θ (rad):", theta)
print("Estimated bending plane φ (rad):", phi)

# 6) Plot a 2D x–z view of the carved voxels and the centerline
fig, ax = plt.subplots()
ax.scatter(points[:,0], points[:,2], marker='.', label='voxels')
ax.plot(centerline[:,0], centerline[:,2], label='centerline')
ax.set_xlabel('x')
ax.set_ylabel('z')
ax.legend()
plt.show()


Hello, World!
